In [ ]:
!pip install accelerate peft bitsandbytes transformers

In [ ]:
!pip install -U datasets

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import torch
print("CUDA:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

CUDA: True
GPU: NVIDIA A100-SXM4-40GB


In [ ]:
import numpy as np
import pandas as pd

print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

NumPy version: 2.0.2
Pandas version: 2.2.2


In [ ]:
!pip install flash-attn --no-build-isolation

In [ ]:
import torch, re, math, random
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm

DATASET

In [ ]:
# Dataset
data_files = {
    "train": "train.jsonl",
    "test": "test.jsonl"
}
from datasets import load_dataset

dataset = load_dataset("bengisucam/LLL_INO-tagged", data_files=data_files, cache_dir=None, download_mode='force_redownload')

# Access the splits directly from the loaded dataset
train_dataset = dataset["train"]
val_dataset = dataset["test"]

print(train_dataset[:2])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.jsonl: 0.00B [00:00, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/139 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25 [00:00<?, ? examples/s]

{'Unnamed: 0': [0, 1], 'Gene1': ['KinC', 'KinD'], 'Gene2': ['Spo0A~P', 'Spo0A~P'], 'Keywords': ['responsible, production', 'responsible, production'], 'Direct dependency relations': ['nmod(responsible, production)', 'nmod(responsible, production)'], 'INO Type': ['regulation of translation', 'regulation of translation'], 'INO ID': ['INO_0000034', 'INO_0000034'], 'Num. Keywords': ['2', '2'], 'Sentence': ['In vivo studies of the activity of four of the kinases, KinA, [PROTEIN1] KinC [/PROTEIN1], KinD (ykvD) and KinE (ykrQ), using abrB transcription as an indicator of [PROTEIN2] Spo0A~P [/PROTEIN2] level, revealed that KinC and KinD were responsible for Spo0A~P production during the exponential phase of growth in the absence of KinA and KinB.', 'In vivo studies of the activity of four of the kinases, KinA, KinC, [PROTEIN1] KinD [/PROTEIN1] (ykvD) and KinE (ykrQ), using abrB transcription as an indicator of [PROTEIN2] Spo0A~P [/PROTEIN2] level, revealed that KinC and KinD were responsible f

In [ ]:
import torch, re, math, random
from collections import Counter
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm

MODEL CONFIG

In [ ]:
MODEL_NAME = "Qwen/Qwen3-30B-A3B-Instruct-2507"
MAX_NEW_TOKENS = 32
BATCH_SIZE = 4
TEMPERATURE = 0.2   # generate'de koşullu set ediyoruz (uyarı olmasın)
TOP_P = 1.0
SEED = 42
random.seed(SEED)

HELPER FUNCTIONS

In [ ]:
def normalize_keyword_set(value):
    if value is None or (isinstance(value, float) and math.isnan(value)):
        return set()
    if isinstance(value, list):
        toks = [str(x).strip().lower() for x in value if str(x).strip()]
    else:
        toks = [t.strip().lower() for t in str(value).split(",") if t.strip()]
    toks = [t for t in toks if t not in {"none", "no", "n/a"}]
    return set(toks)

def parse_model_output_to_set(text):
    if text is None:
        return set()
    line = str(text).strip().splitlines()[0]
    toks = [t.strip().lower() for t in line.split(",") if t.strip()]
    cleaned = []
    for t in toks:
        t = re.sub(r"^(answer|keywords?|key word\(s\)|output)\s*[:\-]\s*", "", t).strip()
        if t:
            cleaned.append(t)
    return set(cleaned)

def micro_prf1(golds, preds):
    tp = fp = fn = 0
    for g, p in zip(golds, preds):
        tp += len(g & p); fp += len(p - g); fn += len(g - p)
    prec = tp/(tp+fp) if (tp+fp) else 0.0
    rec  = tp/(tp+fn) if (tp+fn) else 0.0
    f1   = 2*prec*rec/(prec+rec) if (prec+rec) else 0.0
    return {"precision": prec, "recall": rec, "f1": f1, "tp":tp, "fp":fp, "fn":fn}

def keywords_to_str(value):
    return ", ".join(sorted(normalize_keyword_set(value))) if value is not None else ""

PROMPT

In [ ]:
SYSTEM_MSG = (
    "You are a biomedical IE assistant. Return ONLY the interaction keyword(s) "
    "as a comma-separated list. Do not add extra words."
)
INSTRUCTION = (
    "Identify the interaction keyword(s) that describe the relation between the two proteins "
    "tagged with [PROTEIN1]...[/PROTEIN1] and [PROTEIN2]...[/PROTEIN2] in the sentence. "
    "Return only a comma-separated list (e.g., 'transcription, depends'). If none, output 'none'."
)

def build_chat(sentence, demos=None):
    """
    demos: [(demo_sentence, demo_answer_csv), ...] ya da None
    """
    parts = []
    if demos:
        for s_demo, kw_demo in demos:
            parts.append(f"Example:\nSentence: {s_demo}\nAnswer: {kw_demo}")
    parts.append(
        INSTRUCTION +
        "\nReturn format: just the keywords as a single comma-separated line. "
        "Do not output anything else."
        f"\n\nSentence: {sentence}"
    )
    user_block = "\n\n".join(parts)
    return [
        {"role": "system", "content": SYSTEM_MSG},
        {"role": "user", "content": user_block},
    ]

FEW SHOT HELPER

In [ ]:
def pick_covering_demos(train_dataset, k=5, max_chars=420):
    """
    En sık görülen keyword'leri olabildiğince kapsayan k adet örnek seçer (greedy).
    """
    rows = []
    for i in range(len(train_dataset)):
        ks = normalize_keyword_set(train_dataset[i]["Keywords"])
        rows.append((i, ks))

    freq = Counter()
    for _, ks in rows:
        freq.update(ks)

    chosen, covered = [], set()
    targets = [w for w,_ in freq.most_common()]
    for _ in range(min(k, len(rows))):
        best_i, best_gain = None, -1
        for i, ks in rows:
            if i in chosen:
                continue
            gain = len((ks - covered) & set(targets))
            if gain > best_gain:
                best_i, best_gain = i, gain
        if best_i is None:
            break
        chosen.append(best_i)
        covered |= rows[best_i][1]

    demos = []
    for i in chosen:
        s = train_dataset[i]["Sentence"]
        kw = keywords_to_str(train_dataset[i]["Keywords"])
        if len(s) > max_chars:
            s = s[:max_chars] + " ..."
        demos.append((s, kw))
    return demos

MODEL LOADER

In [ ]:
def load_model_and_tokenizer(model_name=MODEL_NAME):
    bf16_ok = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
    bnb = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16 if bf16_ok else torch.float16,
    )
    tok = AutoTokenizer.from_pretrained(model_name, use_fast=True, trust_remote_code=True)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    tok.padding_side = "left"  # decoder-only generation için doğru

    attn_kwargs = {}
    if torch.cuda.is_available():
        attn_kwargs["attn_implementation"] = "flash_attention_2"  # CPU'da otomatik pas

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb,
        device_map="auto",
        trust_remote_code=True,
        **attn_kwargs
    )
    model.eval()
    return model, tok


GENERATOR

In [ ]:
@torch.no_grad()
def _generate(enc, model, tok):
    gen_kwargs = dict(
        max_new_tokens=MAX_NEW_TOKENS,
        eos_token_id=tok.eos_token_id,
        pad_token_id=tok.pad_token_id,
    )
    if TEMPERATURE and TEMPERATURE > 0:
        gen_kwargs.update(dict(do_sample=True, temperature=TEMPERATURE, top_p=TOP_P))
    else:
        gen_kwargs.update(dict(do_sample=False))
    return model.generate(**enc, **gen_kwargs)

EVAL: ZERO ONE FEW

In [ ]:
@torch.no_grad()
def evaluate(split_dataset, model, tok, demos=None):
    gold_sets, pred_sets, raw_outputs = [], [], []
    inputs = []
    for sent in split_dataset["Sentence"]:
        messages = build_chat(sent, demos=demos)
        # Qwen modelleri için de apply_chat_template uygundur (trust_remote_code=True)
        prompt = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs.append(prompt)

    for i in tqdm(range(0, len(inputs), BATCH_SIZE)):
        batch_texts = inputs[i:i+BATCH_SIZE]
        enc = tok(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=2048).to(model.device)
        out = _generate(enc, model, tok)
        for j in range(out.size(0)):
            gen_ids = out[j, enc["input_ids"].size(1):]
            text = tok.decode(gen_ids, skip_special_tokens=True).strip()
            raw_outputs.append(text)
            pred_sets.append(parse_model_output_to_set(text))

    gold_sets = [normalize_keyword_set(k) for k in split_dataset["Keywords"]]
    scores = micro_prf1(gold_sets, pred_sets)
    return scores, raw_outputs, gold_sets, pred_sets

MAIN RUN

In [ ]:
model, tok = load_model_and_tokenizer()

# ZERO-SHOT
scores_z, raw_z, gold_z, pred_z = evaluate(val_dataset, model, tok, demos=None)
print("\n=== ZERO-SHOT (Qwen3-4B-Instruct-2507) ===")
print({k: round(v,4) if isinstance(v,float) else v for k,v in scores_z.items()})

# ONE-SHOT (k=1)
one_shot = pick_covering_demos(train_dataset, k=1)
scores_1, raw_1, gold_1, pred_1 = evaluate(val_dataset, model, tok, demos=one_shot)
print("\n=== ONE-SHOT (k=1) ===")
print({k: round(v,4) if isinstance(v,float) else v for k,v in scores_1.items()})

# FEW-SHOT (k=5)
few_shot = pick_covering_demos(train_dataset, k=5)
scores_5, raw_5, gold_5, pred_5 = evaluate(val_dataset, model, tok, demos=few_shot)
print("\n=== FEW-SHOT (k=5) ===")
print({k: round(v,4) if isinstance(v,float) else v for k,v in scores_5.items()})

# örnek çıktılar (few-shot)
for i in range(min(5, len(val_dataset))):
    print(f"\nSentence: {val_dataset[i]['Sentence']}")
    print(f"Gold: {gold_5[i]}")
    print(f"Pred: {pred_5[i]} | Raw='{raw_5[i]}'")

Loading checkpoint shards:   0%|          | 0/16 [00:00<?, ?it/s]

100%|██████████| 7/7 [01:50<00:00, 15.76s/it]



=== ZERO-SHOT (Qwen3-4B-Instruct-2507) ===
{'precision': 0.1961, 'recall': 0.2174, 'f1': 0.2062, 'tp': 10, 'fp': 41, 'fn': 36}


100%|██████████| 7/7 [02:33<00:00, 21.98s/it]



=== ONE-SHOT (k=1) ===
{'precision': 0.2464, 'recall': 0.3696, 'f1': 0.2957, 'tp': 17, 'fp': 52, 'fn': 29}


100%|██████████| 7/7 [02:15<00:00, 19.42s/it]


=== FEW-SHOT (k=5) ===
{'precision': 0.3704, 'recall': 0.4348, 'f1': 0.4, 'tp': 20, 'fp': 34, 'fn': 26}

Sentence: In addition to the typical [PROTEIN1] sigmaB [/PROTEIN1]-dependent, stress- and starvation-inducible pattern, [PROTEIN2] yvyD [/PROTEIN2] is also induced in response to amino acid depletion.
Gold: {'inducible', 'dependent'}
Pred: {'none'} | Raw='none'

Sentence: We overproduced and purified sigma(X) from Escherichia coli and demonstrate that in vitro, both sigma(A) and [PROTEIN1] sigma(X) [/PROTEIN1] holoenzymes recognize promoter elements within the sigX-[PROTEIN2] ypuN [/PROTEIN2] control region.
Gold: {'recognize', 'promoter'}
Pred: {'recognize', 'holoenzymes'} | Raw='recognize, holoenzymes'

Sentence: The sigmaB-dependent promoter drives expression of yvyD under stress conditions and after glucose starvation, whereas a [PROTEIN1] sigmaH [/PROTEIN1]-dependent promoter is responsible for [PROTEIN2] yvyD [/PROTEIN2] transcription following amino acid limitation.
Gold: {'